In [81]:
# Write Backend that give Vertical Analysis - Horizontal Analysis, Ratio, Simple Forecast, out of some Financial Statements.
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)

#specify that all rows should be shown
pd.set_option('display.max_rows', None)

Bc2020 = pd.read_excel("C:/Users/Diego/Documents/Red Contable/data entrada/data_entrada_excel_fsm_2020.xlsx", "BC-2020")
Bc2021 = pd.read_excel("C:/Users/Diego/Documents/Red Contable/data entrada/data_entrada_excel_fsm_2021.xlsx", "BC-2021")
Bc2022 = pd.read_excel("C:/Users/Diego/Documents/Red Contable/data entrada/data_entrada_excel_fsm_2022.xlsx", "BC-2022")

Bc2020["Cierre Financiero"] = pd.to_datetime("2020-12-31")
Bc2021["Cierre Financiero"] = pd.to_datetime("2021-12-31")
Bc2022["Cierre Financiero"] = pd.to_datetime("2022-12-31")

Bccombinado = pd.concat([Bc2020, Bc2021, Bc2022])

Bccombinado = Bccombinado.drop("Unnamed: 9", axis=1)

Bccombinado[["Total Débitos", "Total Créditos"]] = Bccombinado[["Total Débitos", "Total Créditos"]].fillna(0).replace(" ", 0).replace("", 0).replace("  ", 0)

Bccombinado["Código de Cuenta"] = Bccombinado["Código de Cuenta"].astype(str)
Bccombinado[["Total Débitos", "Total Créditos"]] = Bccombinado[["Total Débitos", "Total Créditos"]].astype(float)

Bccombinado[["Total Débitos", "Total Créditos"]].sum()

Bccombinado["Monto"] = Bccombinado["Total Débitos"] - Bccombinado["Total Créditos"]

Bccombinado.drop(["Total Débitos", "Total Créditos"], axis=1, inplace=True)


Bccombinado

,Código de Cuenta,Descripción de la cuenta contable,FSLI,TIPO DE CUENTA,TIEMPO,INTERCOMPAÑIA,TAGS,Cierre Financiero,Monto
0,11103,Fondo de caja chica,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,425.000
1,11104,Banco BBVA - moneda nacional,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,119571.200
2,11105,Cuenta de ahorros - Wells fargo,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,52229.950
3,11106,Banco Santander - moneda nacional,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,12948.900
4,11107,Banco Suntrust - US dólares,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,11050.000
5,11108,Banco BBVA - Euros,EFECTIVO,ACTIVO,CORRIENTE,False,NaN,2020-12-31,34850.000
6,11206,Cuentas por cobrar comerciales mayoristas,CUENTAS POR COBRAR,ACTIVO,CORRIENTE,False,NaN,2020-12-31,108696.300
7,11209,Cuentas por cobrar comerciales detallistas,CUENTAS POR COBRAR,ACTIVO,CORRIENTE,False,NaN,2020-12-31,55420.850
8,11211,Cuentas por cobrar otros clientes,CUENTAS POR COBRAR,ACTIVO,CORRIENTE,False,NaN,2020-12-31,19180.060
9,11214,Otras cuentas por cobrar,CUENTAS POR COBRAR,ACTIVO,CORRIENTE,False,NaN,2020-12-31,7211.060


In [82]:
order_ic = ["VENTAS NETAS", "OTROS INGRESOS", "DESCUENTOS", "TOTAL INGRESOS", "COSTO DE VENTA", 
            "GROSS PROFIT", "GASTOS DE ADMINISTRACION", "DEPRECIACION", "DIFERENCIAL CAMBIARIO", 
            "OPERATING PROFIT", "IMPUESTOS", "NET PROFIT"]

def turn_tofsli(balancecomprobacion):
    income_statement = balancecomprobacion[balancecomprobacion["TIPO DE CUENTA"].isin(["INGRESOS", "EGRESOS"])].groupby(["FSLI", "Cierre Financiero"]).sum()
    income_statement = income_statement.reset_index().drop(["TAGS", "INTERCOMPAÑIA"],axis=1)
    income_statement = income_statement.pivot_table(columns="Cierre Financiero", index="FSLI")*-1
    income_statement.loc["TOTAL INGRESOS", :] = income_statement.loc["VENTAS NETAS", :] + income_statement.loc["OTROS INGRESOS", :] + income_statement.loc["DESCUENTOS", :]
    income_statement.loc["GROSS PROFIT", :] = income_statement.loc["TOTAL INGRESOS", :] + income_statement.loc["COSTO DE VENTA", :]
    income_statement.loc["OPERATING PROFIT", :] = income_statement.loc["GROSS PROFIT", :] + income_statement.loc["GASTOS DE ADMINISTRACION", :] + income_statement.loc["DEPRECIACION", :] + income_statement.loc["DIFERENCIAL CAMBIARIO", :]
    income_statement.loc["NET PROFIT", :] = income_statement.loc["OPERATING PROFIT", :] + income_statement.loc["IMPUESTOS", :]
    
    income_statement = income_statement.loc[order_ic, :]
    
    return income_statement


incomestatement = turn_tofsli(Bccombinado)

incomestatement

C:\Users\Diego\AppData\Local\Temp\ipykernel_1168\72663131.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  income_statement = balancecomprobacion[balancecomprobacion["TIPO DE CUENTA"].isin(["INGRESOS", "EGRESOS"])].groupby(["FSLI", "Cierre Financiero"]).sum()


Monto                          
Cierre Financiero          2020-12-31  2021-12-31    2022-12-31
FSLI                                                           
VENTAS NETAS              3707734.670 1381798.000  71509661.000
OTROS INGRESOS             212864.260   25463.000   4461137.000
DESCUENTOS                  16896.300   28173.000    154145.000
TOTAL INGRESOS            3937495.230 1435434.000  76124943.000
COSTO DE VENTA           -2751283.840 -253351.000 -11164799.000
GROSS PROFIT              1186211.390 1182083.000  64960144.000
GASTOS DE ADMINISTRACION  -485112.790 -714319.000 -30824776.000
DEPRECIACION               -21380.050  -64282.000  -2682307.000
DIFERENCIAL CAMBIARIO      -17188.420   17323.000   2287428.000
OPERATING PROFIT           662530.130  420805.000  33740489.000
IMPUESTOS                 -127214.400 -112299.000  -5638674.000
NET PROFIT                 535315.730  308506.000  28101815.000

In [6]:
def vertical_analysis(eeff, comparable, dateofanalysis):
    verticaleeff = pd.DataFrame(eeff.loc[:, ("Monto", dateofanalysis)])
    verticaleeff[f"% of {comparable}"] = verticaleeff.loc[:, ("Monto", dateofanalysis)] / verticaleeff.loc[comparable, ("Monto", dateofanalysis)]  
    return verticaleeff

vertical_analysis(incomestatement, "TOTAL INGRESOS", "2022-12-31")

,Monto,% of TOTAL INGRESOS
,2022-12-31,NaT
FSLI,,
VENTAS NETAS,71509661.000,0.939
OTROS INGRESOS,4461137.000,0.059
DESCUENTOS,154145.000,0.002
TOTAL INGRESOS,76124943.000,1.000
COSTO DE VENTA,-11164799.000,-0.147
GROSS PROFIT,64960144.000,0.853
GASTOS DE ADMINISTRACION,-30824776.000,-0.405
DEPRECIACION,-2682307.000,-0.035


In [77]:
def horizontal_analysis(fs1):
    df = fs1.copy()
    df = df.iloc[:, ::-1]

    for i in range(0, len(df.columns)):
        col_name = str(df.columns[i]) + " % Change"
        previous_period = df.iloc[:, i + 1]
        current_period = df.iloc[:, i]
        pct_change = ((current_period - previous_period) / previous_period) * 100
        df[col_name] = pct_change
    
    df.columns = [d.strftime('%Y-%m-%d') if isinstance(d, pd.Timestamp) else d for d in df.columns]
    df = df.reindex(sorted(df.columns, reverse=True), axis=1)
    return df

#incomestatement = incomestatement.loc[:, ("Monto", )].reset_index()


income_statementh = horizontal_analysis(incomestatement)

income_statementh.drop("2020-12-31 00:00:00 % Change", axis=1)

,2022-12-31 00:00:00 % Change,2022-12-31,2021-12-31 00:00:00 % Change,2021-12-31,2020-12-31
FSLI,,,,,
VENTAS NETAS,5075.117,71509661.000,-62.732,1381798.000,3707734.670
OTROS INGRESOS,17420.076,4461137.000,-88.038,25463.000,212864.260
DESCUENTOS,447.137,154145.000,66.741,28173.000,16896.300
TOTAL INGRESOS,5203.270,76124943.000,-63.544,1435434.000,3937495.230
COSTO DE VENTA,4306.850,-11164799.000,-90.792,-253351.000,-2751283.840
GROSS PROFIT,5395.396,64960144.000,-0.348,1182083.000,1186211.390
GASTOS DE ADMINISTRACION,4215.268,-30824776.000,47.248,-714319.000,-485112.790
DEPRECIACION,4072.719,-2682307.000,200.663,-64282.000,-21380.050
DIFERENCIAL CAMBIARIO,13104.572,2287428.000,-200.783,17323.000,-17188.420
